In [8]:
import argparse
import os
import math
import random
import numpy as np

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torch
import torch.nn as nn
import torch.nn.functional as F

from encoder import Encoder
from attention import Attention
from decoder import Decoder
from generator import Generator

from data_loader import SpeechDataset, Padding, ToTensor

In [7]:
parser = argparse.ArgumentParser(description='RSR-GAN training')
parser.add_argument('--train-manifest', metavar='DIR',
                    help='path to train manifest csv', default='data/train_manifest.csv')
parser.add_argument('--val-manifest', metavar='DIR',
                    help='path to validation manifest csv', default='data/val_manifest.csv')
parser.add_argument('--sample-rate', default=16000, type=int, help='Sample rate')
parser.add_argument('--batch-size', default=20, type=int, help='Batch size for training')
parser.add_argument('--num-workers', default=4, type=int, help='Number of workers used in data-loading')
parser.add_argument('--labels-path', default='labels_dict.json', help='Contains all characters for transcription')
parser.add_argument('--window-size', default=.02, type=float, help='Window size for spectrogram in seconds')
parser.add_argument('--window-stride', default=.01, type=float, help='Window stride for spectrogram in seconds')
parser.add_argument('--window', default='hamming', help='Window type for spectrogram generation')

parser.add_argument('--enc-hid-dim', default=256, type=int, help='Encoder hidden dimension')
parser.add_argument('--dec-hid-dim', default=256, type=int, help='Decoder hidden dimension')
parser.add_argument('--dec-emb-dim', default=256, type=int, help='Decoder embedding dimension')
parser.add_argument('--dropout-rate', default=0.2, type=float, help='Dropout rate')

parser.add_argument('--epochs', default=500, type=int, help='Number of training epochs')
parser.add_argument('--cuda', dest='cuda', action='store_true', help='Use cuda to train model')
parser.add_argument('--lr', '--learning-rate', default=3e-4, type=float, help='initial learning rate')
parser.add_argument('--momentum', default=0.9, type=float, help='momentum')
parser.add_argument('--max-norm', default=400, type=int, help='Norm cutoff to prevent explosion of gradients')
parser.add_argument('--learning-anneal', default=1.1, type=float, help='Annealing applied to learning rate every epoch')
parser.add_argument('--silent', dest='silent', action='store_true', help='Turn off progress tracking per iteration')
parser.add_argument('--checkpoint', dest='checkpoint', action='store_true', help='Enables checkpoint saving of model')
parser.add_argument('--checkpoint-per-batch', default=0, type=int, help='Save checkpoint per batch. 0 means never save')
parser.add_argument('--tensorboard', dest='tensorboard', action='store_true', help='Turn on tensorboard graphing')
parser.add_argument('--log-dir', default='visualize/rsrgan_final', help='Location of tensorboard log')
parser.add_argument('--log-params', dest='log_params', action='store_true', help='Log parameter values and gradients')
parser.add_argument('--id', default='Deepspeech training', help='Identifier for visdom/tensorboard run')
parser.add_argument('--save-folder', default='models/', help='Location to save epoch models')
parser.add_argument('--model-path', default='models/rsrgan_final.pth',
                    help='Location to save best validation model')
parser.add_argument('--continue-from', default='', help='Continue from checkpoint model')
parser.add_argument('--finetune', dest='finetune', action='store_true',
                    help='Finetune the model from checkpoint "continue_from"')

parser.add_argument('--no-shuffle', dest='no_shuffle', action='store_true',
                    help='Turn off shuffling and sample from dataset based on sequence length (smallest to largest)')

#args = parser.parse_args()

_StoreTrueAction(option_strings=['--no-shuffle'], dest='no_shuffle', nargs=0, const=True, default=False, type=None, choices=None, help='Turn off shuffling and sample from dataset based on sequence length (smallest to largest)', metavar=None)

In [4]:
SEQ_LEN = src.size(0)
INPUT_SIZE = src.size(2)
OUTPUT_DIM = 28
ENC_HID_DIM = 256
DEC_HID_DIM = 256 
DEC_EMB_DIM = 256
DROPOUT_RATE = 0.2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

encoder = Encoder(SEQ_LEN, INPUT_SIZE, ENC_HID_DIM, DEC_HID_DIM, DROPOUT_RATE)
attention = Attention(enc_hid_dim=ENC_HID_DIM, dec_hid_dim=DEC_HID_DIM)
decoder = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DROPOUT_RATE, attention)
model = Generator(encoder, decoder, device).to(device)

NameError: name 'src' is not defined

In [13]:
model

Generator(
  (encoder): Encoder(
    (bi_gru1): GRU(241, 256, bidirectional=True)
    (bi_gru2): GRU(768, 256, bidirectional=True)
    (bi_gru3): GRU(768, 256, bidirectional=True)
    (bi_gru4): GRU(768, 256, bidirectional=True)
    (bi_gru5): GRU(768, 256, bidirectional=True)
    (bi_gru6): GRU(768, 256, bidirectional=True)
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (pool): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
    (dropout): Dropout(p=0.2)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=768, out_features=256, bias=True)
    )
    (embedding): Embedding(28, 256)
    (gru): GRU(768, 256)
    (out): Linear(in_features=1024, out_features=28, bias=True)
    (dropout): Dropout(p=0.2)
  )
)

In [11]:
audio_conf = {'window': 'hamming',
              'window_size' : 0.02,
              'window_stride' : 0.01,
              'sampling_rate': 16000}

speech_dataset = SpeechDataset('/media/ai/SpeechRecognition.EN/codebase/DVD/train_manifest.csv', 
                               'labels_dict.json',
                               audio_conf,
                              )
#transform=transforms.Compose([Padding(30000, 50, 'labels_dict.json')]) 

In [13]:
from tqdm import tqdm

In [14]:
max_audio_len = 0
max_txt_len = 0
for i in tqdm(range(len(speech_dataset))):
    sample = speech_dataset[i]
    if sample['signal'].shape[1] > max_audio_len:
        max_audio_len = sample['signal'].shape[1]
        
    if sample['transcript'].shape[0] > max_txt_len:
        max_txt_len = sample['transcript'].shape[0]

max_audio_len, max_txt_len

  0%|          | 1174/265878 [00:03<12:27, 353.98it/s]


ValueError: There aren't any elements to reflect in axis 0 of `array`